In [39]:
import pandas as pd
import numpy as np
from anndata import AnnData
import anndata as ad
import scanpy as sc
import scipy.sparse as sp
import glob
from tqdm import tqdm
from paste import pairwise_align
import ot
import torch
import matplotlib.pyplot as plt
import os

In [27]:
all_index = [25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 147, 149, 151, 153, 155]

In [28]:
df = pd.read_csv('/home/hanyuji/Data/ST_data/macaque_cortex/mmc3_macaque.csv')
df

,Section ID,Monkey,EBZ (mm),Number of cells
0,T155,#1,41.08,17613
1,T153,#1,40.06,42104
2,T151,#1,39.58,53616
3,T149,#1,39.05,50213
4,T147,#1,38.56,37766
...,...,...,...,...
156,T863,#3,-6.01,423181
157,T864,#3,-9.03,441002
158,T866,#3,-12.07,464387
159,T868,#3,-15.04,253554


In [29]:
sub_df = df[df['Monkey'] == '#1']
sub_df['z'] = pd.to_numeric(sub_df['EBZ (mm)'].str.replace(' ', ''), errors='coerce')  # 字符串转换为浮点数

sub_df['gap'] = sub_df['z'].diff()
sub_df.loc[sub_df.index[0], 'gap'] = 0 # 将第一行的'gap'值设为0
sub_df

/tmp/ipykernel_854259/3675401388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['z'] = pd.to_numeric(sub_df['EBZ (mm)'].str.replace(' ', ''), errors='coerce')  # 字符串转换为浮点数
/tmp/ipykernel_854259/3675401388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['gap'] = sub_df['z'].diff()


,Section ID,Monkey,EBZ (mm),Number of cells,z,gap
0,T155,#1,41.08,17613,41.08,0.00
1,T153,#1,40.06,42104,40.06,-1.02
2,T151,#1,39.58,53616,39.58,-0.48
3,T149,#1,39.05,50213,39.05,-0.53
4,T147,#1,38.56,37766,38.56,-0.49
...,...,...,...,...,...,...
114,T136,#1,-17.87,134776,-17.87,-0.63
115,T134,#1,-18.46,259903,-18.46,-0.59
116,T132,#1,-19.17,227914,-19.17,-0.71
117,T128,#1,-19.58,117120,-19.58,-0.41


In [30]:
# 排序
# df2.sort_values(by=2, ascending=False, inplace=True)


In [42]:
# 检查文件存在
directory = '/home/hanyuji/Data/ST_data/macaque_cortex/ST_result/20230503-macaque1-sct'  # replace with your directory path
file_index = sub_df['Section ID'].to_list()

file_existence = {}
for index in file_index:
    file_path = os.path.join(directory, f'total_gene_{index}.type20230503-macaque1-contour2.round.txt')
    file_existence[index] = os.path.exists(file_path)

for file, exists in file_existence.items():
    print(f"'{file}' exists: {exists}")


'T155' exists: True
'T153' exists: True
'T151' exists: True
'T149' exists: True
'T147' exists: True
'T144' exists: True
'T142' exists: True
'T140' exists: True
'T138' exists: True
'T143' exists: True
'T141' exists: True
'T139' exists: True
'T137' exists: True
'T135' exists: True
'T133' exists: True
'T130' exists: True
'T131' exists: True
'T129' exists: True
'T127' exists: True
'T125' exists: True
'T123' exists: True
'T121' exists: True
'T119' exists: True
'T117' exists: True
'T115' exists: True
'T113' exists: True
'T111' exists: True
'T109' exists: True
'T107' exists: True
'T105' exists: True
'T103' exists: True
'T101' exists: True
'T99' exists: True
'T97' exists: True
'T95' exists: True
'T93' exists: True
'T91' exists: True
'T89' exists: True
'T87' exists: True
'T85' exists: True
'T83' exists: True
'T81' exists: True
'T79' exists: True
'T77' exists: True
'T75' exists: True
'T73' exists: True
'T71' exists: True
'T67' exists: True
'T65' exists: True
'T63' exists: True
'T61' exists: True